In [2]:
## Prepare callbacks

In [3]:
import os

In [4]:
%pwd

'd:\\neww\\cnnClassifier\\research'

In [5]:
os.chdir("../")

In [6]:
%pwd

'd:\\neww\\cnnClassifier'

In [7]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [9]:
from cnnClassifier.constants import *
from cnnClassifier.utils import read_yaml, create_directories

In [10]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config

In [11]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

[2025-12-27 01:15:29,196: INFO: utils: NumExpr defaulting to 4 threads.]


In [15]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [17]:
 import time
timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
timestamp

'2025-12-27-01-19-54'

In [18]:
import time
timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
f"tb_logs_at_{timestamp}" 

'tb_logs_at_2025-12-27-01-21-04'

In [19]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
    
except Exception as e:
    raise e

[2025-12-27 01:23:07,520: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-12-27 01:23:07,523: INFO: common: yaml file: params.yaml loaded successfully]
[2025-12-27 01:23:07,525: INFO: common: created directory at: artifacts]
[2025-12-27 01:23:07,528: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2025-12-27 01:23:07,530: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
